<a href="https://colab.research.google.com/github/PA100001/AI/blob/main/Text_matching%20(BERT%20and%20spacy).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_md


In [ ]:
import os
os.kill(os.getpid(), 9)

In [3]:
import pandas as pd
import numpy as np
import spacy
from sklearn.metrics.pairwise import cosine_similarity

# Load the Excel files into pandas DataFrames
df1 = pd.read_excel('/content/2019.xlsx')
df2 = pd.read_excel('/content/2024.xlsx')

# Extract the 'Name' columns
names1 = df1["Name of Polling Station"].astype(str)
names2 = df2["Name"].astype(str)


nlp = spacy.load('en_core_web_md')

# Function to get vector for a name
def get_vector(name):
    return nlp(name).vector

# Create vectors for names
vectors1 = [get_vector(name) for name in names1]
vectors2 = [get_vector(name) for name in names2]

# Compute cosine similarity
similarity_matrix = cosine_similarity(vectors1, vectors2)

##Without Threshold

In [4]:
#  Get the index of the most similar name
matches = np.argmax(similarity_matrix, axis=1)

# Display matches
for i, match_index in enumerate(matches):
    print(f"{names1.iloc[i]} || {names2.iloc[match_index]}")

government sr. sec.school, nawanagar || government primary school, kajad, bhoj kudana
government primary school,shahapur || government primary school, samalutha bhauj kudana
government primary school, khokhra || government primary school, samalutha bhauj kudana
government primary school, gorakhnath || government primary school, samalutha bhauj kudana
government high school, maranwala || government high school, khairwali parwala
government high school, maranwala || government high school, khairwali parwala
government high school, maranwala || government high school, khairwali parwala
government middle school, kona || government primary school, kutbe wala
government middle school, rampur jangi || government secondary school, rampur jangi
government middle school, kholmulla || government secondary school, rampur jangi
government primary school, kholfateh singh || government primary school, badauna kalan
government senior secondary school, nanakpur || government senior secondary school, ba

##Without Threshold

In [7]:
threshold = 0.95

# Match names with confidence
for i, row in enumerate(similarity_matrix):
    match_index = np.argmax(row)
    confidence = row[match_index]
    if confidence > threshold:
        print(f"{names1.iloc[i]} || {names2.iloc[match_index]} CONFIDENCE {confidence:.2f}")
    else:
        continue

government primary school,shahapur || government primary school, samalutha bhauj kudana CONFIDENCE 1.00
government primary school, khokhra || government primary school, samalutha bhauj kudana CONFIDENCE 1.00
government primary school, gorakhnath || government primary school, samalutha bhauj kudana CONFIDENCE 1.00
government high school, maranwala || government high school, khairwali parwala CONFIDENCE 1.00
government high school, maranwala || government high school, khairwali parwala CONFIDENCE 1.00
government high school, maranwala || government high school, khairwali parwala CONFIDENCE 1.00
government middle school, kona || government primary school, kutbe wala CONFIDENCE 0.96
government middle school, rampur jangi || government secondary school, rampur jangi CONFIDENCE 0.96
government middle school, kholmulla || government secondary school, rampur jangi CONFIDENCE 0.96
government primary school, kholfateh singh || government primary school, badauna kalan CONFIDENCE 0.96
government s

#Using BERT

In [8]:
!pip install transformers torch


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [10]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load the Excel files into pandas DataFrames
df1 = pd.read_excel('/content/2019.xlsx')
df2 = pd.read_excel('/content/2024.xlsx')


# Extract the 'Name' columns
names1 = df1["Sl.No. of Polling Station"].astype(str)+df1["Name of Polling Station"].astype(str)
names2 = df2["ID"].astype(str)+df2["Name"].astype(str)

# Only names no ID
# names1 = df1["Name of Polling Station"].astype(str)
# names2 = df2["Name"].astype(str)

# print(names1)
# print(names2)
# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_vector(name):
    inputs = tokenizer(name, return_tensors='pt', truncation=True, padding=True, max_length=128)
    outputs = model(**inputs)
    # Get the embeddings for [CLS] token (first token) which represents the sentence
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

In [11]:
# Create vectors for names
vectors1 = np.vstack([get_bert_vector(name) for name in names1])
vectors2 = np.vstack([get_bert_vector(name) for name in names2])


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2778: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


####Without Threshold

In [13]:
# Compute cosine similarity
similarity_matrix = cosine_similarity(vectors1, vectors2)

# Get the index of the most similar name
matches = np.argmax(similarity_matrix, axis=1)

# Display matches
for i, match_index in enumerate(matches):
    print(i)
    print(f"{names1.iloc[i]} || {names2.iloc[match_index]}")


0
1government sr. sec.school, nawanagar || 3government higher secondary school, navangar
1
2government primary school,shahapur || 2government primary school, shahpur
2
3government primary school, khokhra || 124government primary school, khadkua
3
4government primary school, gorakhnath || 4government primary school, gorakhnath
4
5government high school, maranwala || 197government primary school, badauna kalan
5
6government high school, maranwala || 197government primary school, badauna kalan
6
7government high school, maranwala || 197government primary school, badauna kalan
7
8government middle school, kona || 207government high school, mauli (western part)
8
9government middle school, rampur jangi || 9government secondary school, rampur jangi
9
10government middle school, kholmulla || 10government secondary school, kholmulla
10
11government primary school, kholfateh singh || 124government primary school, khadkua
11
12government senior secondary school, nanakpur || 12government senior s

###With Thresholda


In [16]:
threshold = 0.9

for i, row in enumerate(similarity_matrix):
    match_index = np.argmax(row)
    confidence = row[match_index]
    if confidence > threshold:
        print(f"{names1.iloc[i]} || {names2.iloc[match_index]}  CONFIDENCE {confidence:.2f}")
    else:
        continue

1government sr. sec.school, nawanagar || 3government higher secondary school, navangar  CONFIDENCE 0.90
2government primary school,shahapur || 2government primary school, shahpur  CONFIDENCE 0.96
3government primary school, khokhra || 124government primary school, khadkua  CONFIDENCE 0.95
4government primary school, gorakhnath || 4government primary school, gorakhnath  CONFIDENCE 1.00
9government middle school, rampur jangi || 9government secondary school, rampur jangi  CONFIDENCE 0.98
10government middle school, kholmulla || 10government secondary school, kholmulla  CONFIDENCE 0.95
11government primary school, kholfateh singh || 124government primary school, khadkua  CONFIDENCE 0.94
12government senior secondary school, nanakpur || 12government senior secondary school, nanakpur (east part)  CONFIDENCE 0.96
13government senior secondary school, nanakpur || 12government senior secondary school, nanakpur (east part)  CONFIDENCE 0.96
14government primary school, ramnagar || 14government p